In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-03-07T19:16:48.268177-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

dask   : 2022.2.0
logging: 0.5.1.2
numpy  : 1.21.6
optuna : 2.10.1
xgboost: 1.6.2
pandas : 1.3.5



In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
!nvidia-smi

Thu Mar  7 19:16:58 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   39C    P0    51W / 300W |    774MiB / 32505MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_drop_shap_3_p1fe.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_drop_shap_3_p1fe.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [14]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_8.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_8.db", study_name="five_fold_optuna_xgb_8", direction='maximize')

In [15]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 42% CPU time recently (threshold: 10%)


CPU times: user 20 s, sys: 2.77 s, total: 22.8 s
Wall time: 2min 20s


In [16]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.283903,0.136473,0.923302,logloss,0.018542,0.011778,6,126,binary:logistic,0.825063,gpu_hist,COMPLETE
1,1,0.263736,1.726018,0.426849,logloss,9.969687,0.046564,18,180,binary:logistic,0.676596,gpu_hist,COMPLETE
2,2,0.282267,0.690497,0.379572,logloss,0.010351,0.024958,4,145,binary:logistic,0.696395,gpu_hist,COMPLETE


In [17]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_8.csv', index=False)
df.head(8)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (thre

CPU times: user 50.1 s, sys: 9.04 s, total: 59.2 s
Wall time: 8min 3s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.283903,0.136473,0.923302,logloss,0.018542,0.011778,6,126,binary:logistic,0.825063,gpu_hist,COMPLETE
1,1,0.263736,1.726018,0.426849,logloss,9.969687,0.046564,18,180,binary:logistic,0.676596,gpu_hist,COMPLETE
2,2,0.282267,0.690497,0.379572,logloss,0.010351,0.024958,4,145,binary:logistic,0.696395,gpu_hist,COMPLETE
3,3,0.238986,0.003714,0.660371,logloss,8.657530,0.001036,12,98,binary:logistic,0.903035,gpu_hist,COMPLETE
4,4,0.252780,0.024713,0.938934,logloss,8.534117,0.004252,3,169,binary:logistic,0.783884,gpu_hist,COMPLETE
5,5,0.279865,0.316583,0.633089,logloss,0.092044,0.016304,16,157,binary:logistic,0.796921,gpu_hist,COMPLETE
6,6,0.273388,0.175968,0.472204,logloss,0.002282,0.004419,14,233,binary:logistic,0.720275,gpu_hist,COMPLETE
7,7,0.277887,0.071680,0.451117,logloss,0.001928,0.004581,21,48,binary:logistic,0.520738,gpu_hist,COMPLETE


In [18]:
study.best_trial.params

{'alpha': 0.13647262274095476,
 'colsample_bytree': 0.9233017934750487,
 'eval_metric': 'logloss',
 'lambda': 0.01854244375363198,
 'learning_rate': 0.01177815593792072,
 'max_depth': 6,
 'min_child_weight': 126,
 'objective': 'binary:logistic',
 'subsample': 0.8250632352575504,
 'tree_method': 'gpu_hist'}

In [19]:
df.value.max()

0.2839028728910102

In [20]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_8.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 73% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (thre

CPU times: user 18min 27s, sys: 2min 4s, total: 20min 32s
Wall time: 2h 27min 51s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.283903,0.136473,0.923302,logloss,0.018542,0.011778,6,126,binary:logistic,0.825063,gpu_hist,COMPLETE
1,1,0.263736,1.726018,0.426849,logloss,9.969687,0.046564,18,180,binary:logistic,0.676596,gpu_hist,COMPLETE
2,2,0.282267,0.690497,0.379572,logloss,0.010351,0.024958,4,145,binary:logistic,0.696395,gpu_hist,COMPLETE
3,3,0.238986,0.003714,0.660371,logloss,8.657530,0.001036,12,98,binary:logistic,0.903035,gpu_hist,COMPLETE
4,4,0.252780,0.024713,0.938934,logloss,8.534117,0.004252,3,169,binary:logistic,0.783884,gpu_hist,COMPLETE
5,5,0.279865,0.316583,0.633089,logloss,0.092044,0.016304,16,157,binary:logistic,0.796921,gpu_hist,COMPLETE
6,6,0.273388,0.175968,0.472204,logloss,0.002282,0.004419,14,233,binary:logistic,0.720275,gpu_hist,COMPLETE
7,7,0.277887,0.071680,0.451117,logloss,0.001928,0.004581,21,48,binary:logistic,0.520738,gpu_hist,COMPLETE
8,8,0.282403,0.002871,0.767325,logloss,0.250828,0.010047,22,260,binary:logistic,0.630393,gpu_hist,COMPLETE
9,9,0.282935,0.039333,0.570508,logloss,0.026090,0.010066,20,178,binary:logistic,0.727898,gpu_hist,COMPLETE


In [21]:
df.value.max()

0.28466491859039045

In [22]:
study.best_trial.params

{'alpha': 0.0030359800629075912,
 'colsample_bytree': 0.5140635427666428,
 'eval_metric': 'logloss',
 'lambda': 6.398388932171676,
 'learning_rate': 0.011261729578082292,
 'max_depth': 8,
 'min_child_weight': 71,
 'objective': 'binary:logistic',
 'subsample': 0.9331127630107074,
 'tree_method': 'gpu_hist'}